# 📊 SmartCash: Model Evaluation & Comparison Notebook
Deskripsi: Notebook untuk evaluasi komprehensif dan perbandingan model deteksi mata uang Rupiah
* File: 04_evaluation_comparison.ipynb
* Author: Alfrida Sabar


## 🔍 Tinjauan Umum
Notebook ini dirancang untuk:
- Mengevaluasi performa model deteksi mata uang
- Membandingkan 4 skenario eksperimen
- Menghasilkan visualisasi metrik yang komprehensif
- Memberikan insight mendalam tentang kinerja model

## 🎯 Tujuan Utama
1. Analisis metrik deteksi (akurasi, presisi, recall)
2. Perbandingan waktu inferensi
3. Visualisasi perbandingan model

In [ ]:
# Sel 1: Import Pustaka
import os
import sys
import yaml
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

# Import modul kustom
from utils.logger import SmartCashLogger
from handlers.evaluation_handler import EvaluationHandler
from utils.metrics import MetricsCalculator

# Setup logger
logger = SmartCashLogger(__name__)

# 🗂️ Tahap 2: Konfigurasi Path dan Lingkungan

## 📍 Definisi Lokasi Direktori
- Menyiapkan path untuk konfigurasi, hasil, dan sumber daya
- Memastikan struktur direktori yang konsisten


In [ ]:
# Sel 2: Konfigurasi Path dan Lingkungan


BASE_DIR = Path.cwd()
CONFIG_PATH = BASE_DIR / 'configs' / 'base_config.yaml'
RESULTS_DIR = BASE_DIR / 'results' / 'evaluation'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Muat konfigurasi
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

# 🛠️ Tahap 3: Fungsi Utilitas Evaluasi
## 🔬 Utilitas Evaluasi
Fungsi-fungsi pembantu untuk:
- Memuat hasil eksperimen
- Memproses data
- Mempersiapkan analisis



In [ ]:
# Sel 3: Fungsi Utilitas
def load_model_results(scenario_name):
    """
    Muat hasil training untuk skenario tertentu
    
    Args:
        scenario_name (str): Nama skenario yang akan dimuat
    
    Returns:
        dict: Hasil evaluasi model
    """
    try:
        results_path = RESULTS_DIR / f"{scenario_name}_results.json"
        with open(results_path, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        logger.error(f"❌ Hasil untuk {scenario_name} tidak ditemukan!")
        return None

# 📊 Tahap 4: Evaluasi Komprehensif Skenario
## 🧐 Evaluasi Multi-Skenario
Proses evaluasi yang mencakup:
- 4 skenario eksperimen berbeda
- Analisis mendalam setiap model
- Perbandingan sistematis

In [ ]:
# Sel 4: Evaluasi Komprehensif
def evaluate_scenarios():
    """
    Evaluasi dan bandingkan semua skenario eksperimen
    
    Returns:
        dict: Perbandingan hasil semua skenario
    """
    scenarios = [
        'scenario_1',  # YOLOv5 Default - Variasi Posisi
        'scenario_2',  # YOLOv5 Default - Variasi Pencahayaan
        'scenario_3',  # YOLOv5 EfficientNet - Variasi Posisi
        'scenario_4'   # YOLOv5 EfficientNet - Variasi Pencahayaan
    ]
    
    all_results = {}
    
    for scenario in scenarios:
        logger.start(f"🔍 Mengevaluasi {scenario}")
        
        # Handler evaluasi
        evaluation_handler = EvaluationHandler(
            config_path=str(CONFIG_PATH),
            scenario=scenario
        )
        
        # Jalankan evaluasi
        scenario_results = evaluation_handler.run_evaluation()
        all_results[scenario] = scenario_results
        
        logger.success(f"✅ Evaluasi {scenario} selesai")
    
    return all_results

# 📈 Tahap 5: Visualisasi Perbandingan Metrik
## 📏 Analisis Metrik Performa
Visualisasi perbandingan:
- Akurasi
- Presisi
- Recall
- F1-Score
- mAP (Mean Average Precision)

In [ ]:
# Sel 5: Visualisasi Perbandingan Metrik
def plot_metric_comparison(results):
    """
    Buat visualisasi perbandingan metrik antar skenario
    
    Args:
        results (dict): Hasil evaluasi dari semua skenario
    """
    # Tambahkan mAP ke daftar metrik
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'mAP']
    
    plt.figure(figsize=(15, 6))
    
    # Persiapan data
    data = {metric: [results[scenario]['metrics'][metric] for scenario in results] 
            for metric in metrics}
    
    # Plot
    for i, metric in enumerate(metrics, 1):
        plt.subplot(1, len(metrics), i)
        plt.bar(list(results.keys()), data[metric], color='skyblue', edgecolor='navy')
        plt.title(f'Perbandingan {metric.upper()}')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('Nilai')
        
        # Tambahkan nilai di atas bar
        for j, v in enumerate(data[metric]):
            plt.text(j, v, f'{v:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.suptitle('Perbandingan Metrik Evaluasi Model', fontsize=16)
    plt.savefig(RESULTS_DIR / 'metric_comparison.png', bbox_inches='tight')
    plt.close()

    # Log informasi mAP
    logger.metric("📊 Perbandingan mAP Antar Skenario:")
    for scenario, mAP in zip(results.keys(), data['mAP']):
        logger.metric(f"{scenario}: {mAP:.4f}")

# ⏱️ Tahap 6: Analisis Waktu Inferensi
## 🚀 Performa Komputasional
Fokus pada:
- Waktu inferensi tiap model
- Perbandingan efisiensi komputasional
- Identifikasi model tercepat

In [ ]:
# Sel 6: Analisis Waktu Inferensi
def analyze_inference_times(results):
    """
    Analisis dan visualisasi waktu inferensi
    
    Args:
        results (dict): Hasil evaluasi dari semua skenario
    """
    inference_times = {
        scenario: results[scenario]['metrics']['inference_time'] 
        for scenario in results
    }
    
    plt.figure(figsize=(10, 5))
    plt.bar(inference_times.keys(), inference_times.values())
    plt.title('Perbandingan Waktu Inferensi Model')
    plt.xlabel('Skenario')
    plt.ylabel('Waktu Inferensi (ms)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'inference_time_comparison.png')
    plt.close()

    return inference_times

# 🏁 Tahap 7: Fungsi Utama
## 🔄 Alur Evaluasi Komprehensif
Langkah-langkah utama:
1. Jalankan evaluasi untuk semua skenario
2. Visualisasi perbandingan metrik
3. Analisis waktu inferensi
4. Simpan laporan detail

In [ ]:
# Sel 7: Fungsi Utama
def run_notebook():
    """
    Jalankan proses evaluasi dan perbandingan model
    """
    try:
        logger.start("🚀 Memulai Evaluasi Komprehensif Model")
        
        # Evaluasi skenario
        evaluation_results = evaluate_scenarios()
        
        # Visualisasi perbandingan metrik
        plot_metric_comparison(evaluation_results)
        
        # Analisis waktu inferensi
        inference_times = analyze_inference_times(evaluation_results)
        
        # Simpan laporan
        with open(RESULTS_DIR / 'evaluation_report.json', 'w') as f:
            json.dump({
                'scenario_results': evaluation_results,
                'inference_times': inference_times
            }, f, indent=2)
        
        logger.success("✨ Evaluasi dan analisis selesai!")
        
    except Exception as e:
        logger.error(f"❌ Terjadi kesalahan: {str(e)}")

# Eksekusi notebook
if __name__ == '__main__':
    run_notebook()